In [ ]:
spark_session.stop()

In [56]:
from pprint import pprint
from operator import add

In [11]:
from pyspark.sql import SparkSession

# (8 cores, 16gb per machine) x 5

# New API
spark_session = SparkSession\
        .builder\
        .master("spark://ben-spark-master:7077") \
        .config('spark.executor.memory', '2g') \
        .config('spark.driver.maxResultSize', 0) \
        .appName("common_crawl")\
        .getOrCreate()
#        .config('spark.executor.cores', 2)\


# Old API (RDD)
spark_context = spark_session.sparkContext

rdd = spark_context.wholeTextFiles('/mnt/nfs/ben-spark-master/teaching/crawl/CC-MAIN-2018-03/foo/CC-MAIN-20180317035630-20180317055630-00000.warc.wet/111*').cache()
rdd.count()

11110

In [9]:
print(spark_context.uiWebUrl)

http://ben-spark-master:4040


In [25]:
rdd.take(1)

[('file:/mnt/nfs/ben-spark-master/teaching/crawl/CC-MAIN-2018-03/foo/CC-MAIN-20180317035630-20180317055630-00000.warc.wet/1014',
  'WARC/1.0\r\nWARC-Type: conversion\r\nWARC-Target-URI: http://angelofdeath.info/tag/rivitalo/\r\nWARC-Date: 2018-03-17T04:36:07Z\r\nWARC-Record-ID: <urn:uuid:bcf2b776-885d-40aa-8794-b735e0edcd95>\r\nWARC-Refers-To: <urn:uuid:4180f1a2-bb55-4733-be49-438167fd58ce>\r\nWARC-Block-Digest: sha1:DTMEUXIG655VC55AKIZV4DGBRQC7DNZ3\r\nContent-Type: text/plain\r\nContent-Length: 1725\r\n\r\nrivitalo\nangelofdeath.info\nHome\nAbout\nContact\nRivitalo\n<<== Rivitalo ==>>\nvuokrataan rivitalo Haapavesi\n4 mh Rivitalo Benalmadena asunnossa 3… vuokra rio guadalquivir.\nmyytävät asunnot taka töölö Kuhmo\nKatso myytävät asunnot Helsinki, Taka-Töölö ja niiden esittelyajat tästä com:issa juuri nyt 127 kohdetta tuoteryhmässä alueella töölö, helsinki.\ntrainer 4you Valkeakoski\nBest 4You Y-tunnus: 21796431 Kotipaikka: LOHJA 4k group 28373204 kangasala.\nmyytävät yksiöt jyväskylä 

In [38]:
#file_content = rdd.take(1)[0][1]
#print(file_content.partition('\r\n\r\n')[2])
from operator import add
import re

p = re.compile('WARC-Target-URI: \S+\.com', re.IGNORECASE)

#print(bool(p.search('\nWARC-Target-URI:\n')))

rdd\
.filter(lambda doc: bool(p.search(doc[1])))\
.map(lambda filename_content: filename_content[1].partition('\r\n\r\n')[2])\
.flatMap(lambda t: t.split(' '))\
.flatMap(lambda w: w.split('\n'))\
.map(lambda w: w.strip())\
.map(lambda w: (w,1))\
.reduceByKey(add)\
.takeOrdered(100, key=lambda x: -x[1])
#.take(100)
#.take(10)
#.flatMap(lambda text: text.split(' ')).take(100)

[('►', 81603),
 ('the', 72647),
 ('-', 66229),
 ('to', 53540),
 ('and', 53526),
 ('(1)', 52009),
 ('a', 49329),
 ('of', 44974),
 ('de', 40495),
 ('in', 31183),
 ('', 23897),
 ('for', 22387),
 ('|', 21020),
 ('(2)', 20626),
 ('I', 19484),
 ('&', 19408),
 ('The', 19392),
 ('is', 19092),
 ('on', 17187),
 ('with', 14468),
 ('by', 14388),
 ('that', 13593),
 ('you', 13114),
 ('at', 13035),
 ('(3)', 12892),
 ('1', 11271),
 ('la', 10538),
 ('que', 10457),
 ('(4)', 10377),
 ('are', 9660),
 ('this', 9634),
 ('en', 9117),
 ('your', 9105),
 ('it', 9037),
 ('2', 8990),
 ('as', 8872),
 ('from', 8658),
 ('be', 8642),
 ('or', 8551),
 (':', 8449),
 ('do', 8160),
 ('e', 7771),
 ('(5)', 7730),
 ('May', 7678),
 ('was', 7498),
 ('have', 7487),
 ('/', 7418),
 ('A', 7365),
 ('3', 7325),
 ('ago', 6985),
 ('y', 6870),
 ('my', 6775),
 ('–', 6404),
 ('4', 6234),
 ('not', 6083),
 ('videos', 6030),
 ('(6)', 5982),
 ('no', 5962),
 ('Home', 5892),
 ('March', 5777),
 ('0', 5771),
 ('5', 5737),
 ('o', 5425),
 ('2017',

In [6]:
rdd.map(lambda filename_content: filename_content[1].split('WARC/1.0')).cache().take(1)

KeyboardInterrupt: 

In [ ]:
rdd.take(1)[0]